In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

## archives.gov electoral vote scraper

In [10]:
def alfa(x): return ''.join([c for c in x if c.isalpha() or c==' '])

In [11]:
elec_vote_map = dict()

if False:
    for y in range(1976, 2021, 4):
        uri = f'https://www.archives.gov/electoral-college/{y}'
        r = requests.get(uri)
        states = bs(r.text, 'html.parser').findAll('tbody')[1].findAll('tr')
        for i in range(2,53):
            cell_list = states[i].findAll('td')

            s = alfa(cell_list[0].text).upper()
            ev = int(cell_list[1].text)
            elec_vote_map[(y,s)] = ev
            pickle.dump(elec_vote_map, open('datasets/elec_vote_map.pkl', 'wb'))
else:
    elec_vote_map = pickle.load(open('datasets/elec_vote_map.pkl', 'rb'))

In [13]:
df = pd.read_csv('datasets-clean/xxx-fitted-values.csv')
df = df[['year', 'state', 'y_votes_percent', 'y_hat_fr', 'y_hat_ds', ]]

df['y_err_fr'] = abs(df.y_votes_percent-df.y_hat_fr)
df['y_err_ds'] = abs(df.y_votes_percent-df.y_hat_ds)

for i in df.index:
    y = df.at[i, 'year']
    s = df.at[i, 'state']

    df.at[i, 'y_ev'] = elec_vote_map[(y,s)] if df.at[i, 'y_votes_percent'] > 50 else 0

    df.at[i, 'y_fr'] = elec_vote_map[(y,s)] if df.at[i, 'y_hat_fr'] > 50 else 0
    df.at[i, 'y_ds'] = elec_vote_map[(y,s)] if df.at[i, 'y_hat_ds'] > 50 else 0

    df.at[i, 'fr_spill'] = df.at[i, 'y_fr'] != df.at[i, 'y_ev']
    df.at[i, 'ds_spill'] = df.at[i, 'y_ds'] != df.at[i, 'y_ev']

In [14]:
df[df.fr_spill].sort_values(['year', 'y_err_fr', ], ascending=False)

,year,state,y_votes_percent,y_hat_fr,y_hat_ds,y_err_fr,y_err_ds,y_ev,y_fr,y_ds,fr_spill,ds_spill
59,2020,COLORADO,56.938320,48.308285,52.496506,8.630035,4.441814,9.0,0.0,9.0,True,False
459,2020,VIRGINIA,55.154686,49.112061,51.311314,6.042625,3.843372,13.0,0.0,13.0,True,False
29,2020,ARIZONA,50.156830,44.180939,49.107407,5.975891,1.049423,11.0,0.0,0.0,True,True
149,2020,IOWA,45.816738,51.412533,54.086185,5.595795,8.269447,0.0,6.0,6.0,True,True
289,2020,NEW HAMPSHIRE,53.748348,49.141781,51.871357,4.606567,1.876991,4.0,0.0,4.0,True,False
99,2020,GEORGIA,50.119331,45.617531,45.552433,4.501800,4.566898,16.0,0.0,0.0,True,True
279,2020,NEVADA,51.223118,49.839470,52.795475,1.383648,1.572357,6.0,0.0,6.0,True,False
148,2016,IOWA,44.936451,50.171982,49.240738,5.235531,4.304287,0.0,6.0,0.0,True,False
458,2016,VIRGINIA,52.826378,47.762558,46.408577,5.063820,6.417801,13.0,0.0,0.0,True,True
288,2016,NEW HAMPSHIRE,50.197029,48.489655,51.875206,1.707374,1.678177,4.0,0.0,4.0,True,False


In [15]:
df[df.ds_spill].sort_values(['year', 'y_err_ds', ], ascending=False)

,year,state,y_votes_percent,y_hat_fr,y_hat_ds,y_err_fr,y_err_ds,y_ev,y_fr,y_ds,fr_spill,ds_spill
149,2020,IOWA,45.816738,51.412533,54.086185,5.595795,8.269447,0.0,6.0,6.0,True,True
99,2020,GEORGIA,50.119331,45.617531,45.552433,4.501800,4.566898,16.0,0.0,0.0,True,True
29,2020,ARIZONA,50.156830,44.180939,49.107407,5.975891,1.049423,11.0,0.0,0.0,True,True
458,2016,VIRGINIA,52.826378,47.762558,46.408577,5.063820,6.417801,13.0,0.0,0.0,True,True
58,2016,COLORADO,52.683334,50.002899,48.051826,2.680435,4.631508,9.0,9.0,0.0,False,True
278,2016,NEVADA,51.293709,49.875954,47.764523,1.417755,3.529186,6.0,0.0,0.0,True,True
218,2016,MICHIGAN,49.882332,51.151192,52.377075,1.268860,2.494743,0.0,16.0,16.0,True,True
277,2012,NEVADA,53.407543,50.126789,49.864796,3.280754,3.542747,6.0,6.0,0.0,False,True
287,2012,NEW HAMPSHIRE,52.833752,51.171474,49.728977,1.662278,3.104775,4.0,4.0,0.0,False,True
457,2012,VIRGINIA,51.967377,49.814861,48.954010,2.152516,3.013367,13.0,0.0,0.0,True,True


In [17]:
# solo sulla percentuale, non sulla assegnazione
np.sqrt(mean_squared_error(df.y_votes_percent, df.y_hat_fr)), np.sqrt(mean_squared_error(df.y_votes_percent, df.y_hat_ds))

(4.206740074727153, 3.5125992180285515)

In [19]:
tab = df.groupby('year')['y_ev', 'y_fr', 'y_ds',].sum()
tab.y_ev += 3
tab.y_fr += 3
tab.y_ds += 3
tab.y_ev = tab.y_ev.astype(int)
tab.y_fr = tab.y_fr.astype(int)
tab.y_ds = tab.y_ds.astype(int)
tab

/tmp/ipykernel_1101/1095840184.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tab = df.groupby('year')['y_ev', 'y_fr', 'y_ds',].sum()


,y_ev,y_fr,y_ds
year,,,
1984,13,73,7
1988,112,220,186
1992,370,367,384
1996,379,394,405
2000,267,203,246
2004,252,238,222
2008,364,316,370
2012,332,290,262
2016,233,262,221
